In [1]:
!pip install transformers datasets --quiet

In [2]:
!pip install --upgrade datasets fsspec --quiet

In [3]:
# hyperparameter tuning
!pip install optuna --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from datasets import disable_caching
import fsspec
import os
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, DataCollatorWithPadding, AutoTokenizer, EarlyStoppingCallback
import numpy as np
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
disable_caching()
print(fsspec.__version__)

2025.3.0


In [ ]:
raw_dataset= load_dataset("glue","mrpc")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_dataset["train"][0,1,2]

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .'],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale ."],
 'label': [1, 0, 1],
 'idx': [0, 1, 2]}

In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

In [ ]:
tokenized_dt= raw_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
data_collator= DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


training_args = TrainingArguments(
    output_dir="test-trainer",
    num_train_epochs=3
   )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dt["train"],
    eval_dataset=tokenized_dt['validation'],
    tokenizer=tokenizer,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2474377437.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
500,0.498400
1000,0.293400


TrainOutput(global_step=1377, training_loss=0.3295960990778328, metrics={'train_runtime': 119.0621, 'train_samples_per_second': 92.422, 'train_steps_per_second': 11.565, 'total_flos': 203961502572816.0, 'train_loss': 0.3295960990778328, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6903791427612305,
 'eval_runtime': 1.1451,
 'eval_samples_per_second': 356.313,
 'eval_steps_per_second': 44.539,
 'epoch': 3.0}

In [ ]:
predictions = trainer.predict(tokenized_dt["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
preds=np.argmax(predictions.predictions,axis=-1)
print(preds)

[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0
 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1
 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 0
 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1
 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 0
 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 1]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_preds):
    logits = eval_preds.predictions
    preds = np.argmax(logits, axis=-1)  # Convert logits to predicted class indices
    labels = eval_preds.label_ids

    metric1 = accuracy_score(labels, preds)
    metric2 = precision_score(labels, preds)
    metric3 = recall_score(labels, preds)
    metric4 = f1_score(labels, preds)
    return {"accuracy": metric1, "precision": metric2, "recall": metric3, "f1": metric4}


In [ ]:
trainer_2 = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dt["train"],
    eval_dataset=tokenized_dt['validation'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/tmp/ipython-input-4074095883.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_2 = Trainer(


In [ ]:
trainer_2.evaluate()

{'eval_loss': 0.6903791427612305,
 'eval_model_preparation_time': 0.0015,
 'eval_accuracy': 0.8455882352941176,
 'eval_precision': 0.839622641509434,
 'eval_recall': 0.956989247311828,
 'eval_f1': 0.8944723618090452,
 'eval_runtime': 1.0158,
 'eval_samples_per_second': 401.662,
 'eval_steps_per_second': 50.208}

In [ ]:
# Optuna for model
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2,
        output_attentions=False,
        output_hidden_states=False,
    )


In [ ]:
# for each hyperparameter
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-5, 5e-5, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
    }


In [ ]:
trainer_3 = Trainer(
    model_init= model_init,
    args=training_args,
    train_dataset= tokenized_dt["train"],
    eval_dataset= tokenized_dt["validation"],
    data_collator= data_collator,
    compute_metrics= compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_run= trainer_3.hyperparameter_search(
    direction="maximize",
    n_trials=3,
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2025-08-07 15:03:43,857] A new study created in memory with name: no-name-a0933a98-ba2a-4ffc-bf8a-c268ae380b2f
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.406300


[I 2025-08-07 15:05:34,997] Trial 0 finished with value: 0.8934010152284264 and parameters: {'learning_rate': 3.100852453421338e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 3}. Best is trial 0 with value: 0.8934010152284264.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


[I 2025-08-07 15:07:02,102] Trial 1 finished with value: 0.8946459412780656 and parameters: {'learning_rate': 2.3930063747492778e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 0.8946459412780656.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


[I 2025-08-07 15:08:51,593] Trial 2 finished with value: 0.8904109589041096 and parameters: {'learning_rate': 3.09958742464868e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 4}. Best is trial 1 with value: 0.8946459412780656.


In [ ]:
training_arg_final = TrainingArguments(
    output_dir="test-trainer",
    num_train_epochs=best_run.hyperparameters["num_train_epochs"],
    per_device_train_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    learning_rate=best_run.hyperparameters["learning_rate"],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer_4 = Trainer(
    model=model,
    args=training_arg_final,
    train_dataset=tokenized_dt["train"],
    data_collator = DataCollatorWithPadding(tokenizer),
    eval_dataset=tokenized_dt["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer_4.train()

Step,Training Loss


TrainOutput(global_step=345, training_loss=0.05483538862587749, metrics={'train_runtime': 88.7024, 'train_samples_per_second': 124.055, 'train_steps_per_second': 3.889, 'total_flos': 226895955866688.0, 'train_loss': 0.05483538862587749, 'epoch': 3.0})

In [ ]:
# showed overfitting
trainer_4.evaluate()

{'eval_loss': 0.8989779949188232,
 'eval_model_preparation_time': 0.0023,
 'eval_accuracy': 0.8284313725490197,
 'eval_precision': 0.8255451713395638,
 'eval_recall': 0.9498207885304659,
 'eval_f1': 0.8833333333333333,
 'eval_runtime': 0.9868,
 'eval_samples_per_second': 413.443,
 'eval_steps_per_second': 51.68}

In [ ]:
# We try to solve overfitting by earlystop and L2

def hp_space_2(trial):
    return {
      "learning_rate": trial.suggest_float("learning_rate", 2e-5, 5e-5, log=True),
      "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8,16,32]),
      "num_train_epochs": trial.suggest_int("num_train_epochs", 2,5),
      "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
    }

In [ ]:
trainer_hp2 = Trainer(
  model_init     = model_init,
  args           = training_args,
  train_dataset  = tokenized_dt["train"],
  eval_dataset   = tokenized_dt["validation"],
  data_collator  = DataCollatorWithPadding(tokenizer),
  compute_metrics= compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_run2 = trainer_hp2.hyperparameter_search(
  direction="maximize",
  n_trials=3,
  hp_space=hp_space_2,
  compute_objective=lambda metrics: metrics["eval_f1"],
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2025-08-07 15:34:04,779] A new study created in memory with name: no-name-b5e3bda9-be3b-43d3-bdc2-158145cd5d75
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.503900
1000,0.323500
1500,0.185700
2000,0.115000


[I 2025-08-07 15:38:34,253] Trial 0 finished with value: 0.8949152542372881 and parameters: {'learning_rate': 2.323906804171938e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.1806779271813365}. Best is trial 0 with value: 0.8949152542372881.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.427000


[I 2025-08-07 15:40:35,773] Trial 1 finished with value: 0.8957264957264958 and parameters: {'learning_rate': 2.3092053238158384e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.21632510271940797}. Best is trial 1 with value: 0.8957264957264958.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.497700
1000,0.305900
1500,0.161500
2000,0.077300


[I 2025-08-07 15:44:53,968] Trial 2 finished with value: 0.8941176470588236 and parameters: {'learning_rate': 3.530561343413613e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 5, 'weight_decay': 0.2438719105641908}. Best is trial 1 with value: 0.8957264957264958.


In [ ]:
best_args =TrainingArguments(
  output_dir="test-trainer",
  eval_strategy="epoch",
  save_strategy="epoch",
  learning_rate=best_run2.hyperparameters["learning_rate"],
  per_device_train_batch_size=best_run2.hyperparameters["per_device_train_batch_size"],
  num_train_epochs=best_run2.hyperparameters["num_train_epochs"],
  load_best_model_at_end=True,
  metric_for_best_model="eval_f1",
  greater_is_better=True,
  weight_decay=best_run2.hyperparameters["weight_decay"],
  warmup_ratio=0.1,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
final_trainer = Trainer(

  model_init     = model_init,
  args           = best_args,
  train_dataset  = tokenized_dt["train"],
  eval_dataset   = tokenized_dt["validation"],
  data_collator  = DataCollatorWithPadding(tokenizer),
  compute_metrics= compute_metrics,
  callbacks      = [EarlyStoppingCallback(early_stopping_patience=3)]
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
final_trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.385931,0.833333,0.872792,0.885305,0.879004
2,No log,0.338383,0.860294,0.862745,0.946237,0.902564
3,0.437900,0.384427,0.850490,0.853896,0.942652,0.896082


TrainOutput(global_step=690, training_loss=0.37941413547681724, metrics={'train_runtime': 155.9857, 'train_samples_per_second': 70.545, 'train_steps_per_second': 4.423, 'total_flos': 215773868887344.0, 'train_loss': 0.37941413547681724, 'epoch': 3.0})

In [ ]:
# epoch 2 has the best weights, we saved best model at the end = true
testing = final_trainer.evaluate(tokenized_dt["test"])
print(testing)

{'eval_loss': 0.39531487226486206, 'eval_accuracy': 0.831304347826087, 'eval_precision': 0.8548922056384743, 'eval_recall': 0.8988666085440279, 'eval_f1': 0.8763280917977051, 'eval_runtime': 4.5511, 'eval_samples_per_second': 379.033, 'eval_steps_per_second': 47.462, 'epoch': 3.0}
